# Zoom simulation computational cost estimator

Use with caution.  c_s and v_s are educated estimates.  Note that this assumes that the time steps are really constrained by the c_s and v_s given; the real average time step is somewhat lower than this, but my goal here is to actually give sensible limits so we don't ask for too little computational time. 

### Note: You must ensure that Max_level and fraction_cells_refined agree!

See the comments below for more information, but fraction_cells_refined must have the right number of elements!

In [ ]:
Lbox = 25.0e+3 / 0.7 # 25 Mpc in kpc (the / 0.7 cleans out the factor of h)
N_rg = 512  # root grid num cells

# fraction of box (per dimension) that is zoomed in to the level described below (Zoom_level)
# note: the volume of the refined region will be (Lbox*zoom_fraction)**3
zoom_fraction = 0.05

# Level that we zoom into, so the dm particle masses are effectively N_rg * 2**Zoom_level
# note: Zoom_level is the level of what Enzo calls a MustRefineRegion or the level of
# MustRefineParticles
Zoom_level = 4  

# assume that the most-zoomed region is padded by this factor (>1) as we go from Zoom_level down to 1
zoom_padding = 1.2

Max_level = 9  # maximum level of refinement within the zoomed region

# Estimate of fraction of cells _per_level_ refined beyond Zoom_level
# at each level beyond Zoom_level.
# This represents an approximation to various refinement schemes.
# Value for each level should be 0 < f <= 1
# 0.125 = Lagrangian-like refinement, 1.0 = complete refinement.
# Should go from Zoom_level + 1 through Max_level

# approximates a volume with a nested refinement region
fraction_cells_refined = [0.125, 0.125, 0.5, 1.0, 0.125]

# Lagrangian-like
#fraction_cells_refined = [0.125]*(Max_level-Zoom_level)

if len(fraction_cells_refined) != Max_level - Zoom_level:
    print("\n\n\n***You don't have the right number of elements in fraction_cells_refined!!!!!\n\t\t\t\texpected, actual:", Max_level - Zoom_level, len(fraction_cells_refined),"\n\n")
    exit(123)
    
# c_s and v_s are sound speed and peculiar gas speed in typical large halos in the box
c_s = 100.0 * 1.e5  # 100 km/s in cm/s
v_s = 250.0 * 1.e5  #  

# assume that this cubed is the number of root grid cells cubed per compute core (16 or so probably makes
# sense for highly zoomed calculations, so 16**3 cells total on the root grid - that is a lot more at
# the highly refined grids!
cells_core = 16

# this is Enzo's approximate speed on Pleiades with All The Physics, and thus we assume Enzo-E's
cell_updates_core_second = 5.0e+3

# at what redshift do you want this simulation to stop?
simulation_end_redshift = 2.0

# A number >= 1 that takes into account miscellaneous overhead such as ghost zones,
# over-refinement, etc.  A number of around 2 is probably sensible.
safety_factor = 2

# number of data outputs (required for data volume estimate)
number_of_data_outputs = 100

print("Root grid dx:  {:.3f} com. Kpc ".format(Lbox/N_rg))
print("Zoom level dx: {:.3f} com. Kpc (L_zoom = {:d})".format(Lbox/N_rg/2**Zoom_level, Zoom_level))
print("Max level dx:  {:.3f} com. Kpc (L_max = {:d})".format(Lbox/N_rg/2**Max_level, Max_level))


In [ ]:
from astropy.cosmology import WMAP9 as cosmo

simulation_end_time = (cosmo.age(simulation_end_redshift)).value  # simulation end time in gyr 

print("simulation end time {:.2f} Gyr".format(simulation_end_time))

In [ ]:
# some useful conversions

Lbox_cm = Lbox * 3.08e21 # box in cm
seconds_per_yr = 365.25*86400
seconds_per_myr = 1.0e6 * seconds_per_yr
universe_age = 13.8e9 * seconds_per_yr

simulation_end_time_seconds = simulation_end_time * 1.0e9 * seconds_per_yr # simulation end time in gyr

In [ ]:
dx_rg = Lbox_cm / N_rg     # root grid dx (in cm)

dt_rg = 0.2 * dx_rg / (c_s + v_s)   # estimate of root grid timestep in seconds (hydro courant time)

dx_zoom = dx_rg / 2**Zoom_level  # cell size at max level in cm

dx_max = dx_rg / 2**Max_level  # cell size at max level in cm

dt_max = dt_rg / 2**Max_level  # timestep at max level in seconds


# number of root grid timesteps
num_rg_ts = simulation_end_time_seconds / dt_rg

# number of max level timesteps (should be much larger than the root grid)
num_maxlev_ts = simulation_end_time_seconds / dt_max

print("Root grid timestep {:.2e} Myrs".format(dt_rg/seconds_per_myr))
print("Max level timestep {:.2e} kyrs (L_max = {:d})".format(dt_max*1.0e3/seconds_per_myr,Max_level))

print("num RG timesteps per simulation: {:.2f}".format(num_rg_ts))
print("num max level timesteps per simulation: {:.2f}".format(num_maxlev_ts))

# Calculate number of cells per level

Also print out some other useful information, for future reference.

In [ ]:
import numpy as np

# array of levels (will be handy later)
level = np.linspace(0,Max_level,Max_level+1,dtype='int')

# number of cells in each dimension (on average) we expect to have at a given level
# this is useful to give a sense of box size, etc.
cells_edge = np.zeros_like(level)

# total number of cells per level - will be cells_edge**3
cells_level = np.zeros_like(level)

# root grid cells per edge should be the size of the root grid
cells_edge[0] = N_rg

# calculate the number of cells per edge at Zoom_level
# recall that Zoom_level is the level of our MustRefineRegion
cells_edge[Zoom_level] = (int) (zoom_fraction*N_rg*2.0**Zoom_level)

# work backwards from Zoom_level to the root grid to estimate number of cells.
# level L-1 has 1/2 the cells of level L per edge if it occupies the same volume, 
# plus multiply by padding factor
for thislev in range(Zoom_level-1,0,-1):
    cells_edge[thislev] = 0.5**(Zoom_level-thislev) * zoom_padding**(Zoom_level-thislev) * cells_edge[Zoom_level]

# work upwards from Zoom level to the maximum refinement level, using the list
# of user-supplied fractions of the number of cells at that level that are refined to
# estimate the number of cells.  Recall that every level we go up (at a given total volume of
# cells) doubles the number of cells per dimension.
for thislev in range(Zoom_level+1,Max_level+1):
    cells_edge[thislev] = (fraction_cells_refined[thislev-Zoom_level-1])**(1./3.)*cells_edge[thislev-1]*2
    #print(thislev, thislev-Zoom_level-1,cells_edge[thislev])

# use cells_edge to calculate cells_level.  We're implicitly assuming cubes.
cells_level = cells_edge**3


print("level\tcells/edge\ttot. cells")

for i in range(Max_level+1):
    print(i, "\t",cells_edge[i],"\t\t{:,}".format(cells_level[i]) )
    
print("\ntotal cells in simulation: {:,}\n".format(cells_level.sum()))

## Calculate grid level information (timesteps, volumes, etc.)

(useful for intuition, etc. mostly)

In [ ]:
# grid cell size per level
dx_level = np.zeros_like(level,dtype='float64')

# time step per level
dt_level = np.zeros_like(level,dtype='float64')

# number of time steps per level (needed for later!)
timesteps_level = np.zeros_like(level,dtype='int')

# if all the cells in a given level were a cube, what would that cube's edge length be?
length_edge_level = np.zeros_like(level,dtype='float64')

# what is the total volume of cells we expect at a given level
total_volume_level = np.zeros_like(level,dtype='float64')

# calculate root grid information!
dx_level[0] = Lbox / N_rg   # in kpc
dt_level[0] = dt_rg         # in seconds
timesteps_level[0] = simulation_end_time_seconds / dt_rg  # unitless
length_edge_level[0] = Lbox  # in kpc
total_volume_level[0] = Lbox**3  # in kpc^3

# now do it for all of the levels from 1 - Max_level
for i in range(1,Max_level+1):
    
    dx_level[i] = dx_level[0] / 2**i
    dt_level[i] = dt_level[0] / 2**i
    
    timesteps_level[i] = (int) (simulation_end_time_seconds / dt_level[i])
    
    length_edge_level[i] = cells_edge[i]*dx_level[i]
    
    total_volume_level[i] = length_edge_level[i]**3

# print all that hard-earned information out for users to stare at in awe
print("\n\nlevel \tdx (com. kpc)\tdt (kyr)\ttime steps\tvol. edge length(com. kpc)\t tot. vol. (com. kpc^3)")
for i in range(0,Max_level+1):
    
    print(i,"\t{:.3f}".format(dx_level[i]),
          "\t\t{:.2e}".format(dt_level[i]/seconds_per_yr/1.e3),
          "\t{:,}".format(timesteps_level[i]),
          "\t\t{:.2f}".format(length_edge_level[i]),
          "\t\t{:e}".format( total_volume_level[i] ) )
    
print("\n\n")

## Conservative estimate

This is extremely conservative, and represents a high upper bound to the estimated simulation runtime.  It assumes universal time steps for the simulation - namely, all time steps are at the timestep of the max level of refinement.  **This represents the worst-case scenario**, if Enzo-E's local causality-preserving time steps are not ready by the time this simulation is run.

In [ ]:
# total number of cells in simulation
total_cells = cells_level.sum()

# total cell updates per max level timestep (which will dominate simulation time)
total_cell_updates = total_cells * timesteps_level[-1]

# multiply in cell safety factor (taking into account ghost zones, etc.)
total_cell_updates *= safety_factor

# total core hours: cell updates / (cell updates per core-hour)
total_core_hours = total_cell_updates / (cell_updates_core_second * 3600.0) 

print("{:e} total cells".format(total_cells))

print("{:e} total cell updates".format(total_cell_updates))

print("{:e} total core hours".format(total_core_hours))

# crude estimate of number of cores we might use
cores = (N_rg//cells_core)**3

# what are the actual number of cells per core we will probably have?
total_cells_core_edge = int(( total_cells / ( cores) )**(1./3.))

# print some info for the user
print("number of cores needed: {:,}".format(cores), "at", 
      str(cells_core)+"^3 root grid cells/core", 
      "(~{:d}^3 total cells per core)".format(total_cells_core_edge))
print("total wall clock hours:  {:.2f}".format(total_core_hours/cores))
print("total wall clock months:  {:.2f}".format(total_core_hours/cores/24./30.))


## Optimistic estimate

This is an estimate that should be taken as a lower bound to the estimated simulation runtime.  It assumes that each grid takes its own locally-determined time step.  **This represents the best-case scenario,** assuming Enzo-E's local causality-preserving timesteps are READY by the time the simulation is run.

In [ ]:
total_cell_updates = (cells_level*timesteps_level).sum()

# multiply in cell safety factor (taking into account ghost zones, etc.)
total_cell_updates *= safety_factor

# total core hours: cell updates / (cell updates per core-hour)
total_core_hours = total_cell_updates / (cell_updates_core_second * 3600.0)

print("{:e} total cells".format(total_cells))

print("{:e} total cell updates".format(total_cell_updates))

print("{:e} total core hours".format(total_core_hours))

# crude estimate of number of cores we might use
cores = (N_rg//cells_core)**3

# what are the actual number of cells per core we will probably have?
total_cells_core_edge = int(( total_cells / ( cores) )**(1./3.))

# print some info for the user
print("number of cores needed: {:,}".format(cores), "at", 
      str(cells_core)+"^3 root grid cells/core", 
      "(~{:d}^3 total cells per core)".format(total_cells_core_edge))
print("total wall clock hours:  {:.2f}".format(total_core_hours/cores))
print("total wall clock months:  {:.2f}".format(total_core_hours/cores/24./30.))


# Data volume estimation

In [ ]:
bytes_per_float = 8 # double precision

# density, total energy, internal energy, 3*velocity, 3*mag field, 6 metal species + "metal",
# 6 primordial species (atomic H, He, and e-), grav. potential
num_fields = 24 

# total_cells includes all levels _ safety factor
data_volume_per_output = total_cells * bytes_per_float * num_fields 

print("each output is: {:.3f} TB".format(data_volume_per_output/1.e12))

data_volume_total = data_volume_per_output*number_of_data_outputs

print("total raw data volume is: {:.3f} TB".format(data_volume_total/1.e12))

print("compressed data volume is:  {:.3f} TB".format(data_volume_total/1.e12/5))